#Imports

In [1]:
import os
import tarfile
import numpy as np

In [1]:
class model:
  def __init__(self, learning_rate):
    self.index_dict = {} #dictionary of words:index in bag of words vector
    #get reviews. add to init when done.
    get_reviews('txt_sentoken/neg', 'txt_sentoken/pos')
    #read_tar()
    neg_path = 'txt_sentoken/neg'
    pos_path = 'txt_sentoken/pos'
    neg_files = os.listdir(neg_path)
    pos_files = os.listdir(pos_path)

    neg_rev, neg_sent, neg_words = self.get_data(neg_files, neg_path, -1)
    pos_rev, pos_sent, pos_words = self.get_data(pos_files, pos_path, 1)
    #form word bag, X_raw, and prediction sets
    words = neg_words|pos_words
    X_raw = np.concatenate((neg_rev,pos_rev),axis=0)
    y = np.concatenate((neg_sent,pos_sent),axis=0)
    sorted_words = sorted(words)
    
  def fit(self, X, y):
    """Does the training. X is a matrix with one data point per row, while y is flat."""
    X = X.ravel()
    best_score = self.score(X,y)
    self.loss.append(best_score)
    early_stop,iter_wo_improve = 20,0
    
    for i in range(self.n_max_iterations):
      self._fit(X,y)
      if best_score == self.loss[-1]: #if the new theta isn't better than last, then increment iter_wo_improve.
        iter_wo_improve += 1
      else:
        best_score = self.loss[-1]
      if iter_wo_improve >= early_stop: #implement learning rate scheduler here
        break
  
  def _fit(self, X, y): 
    """Internal method that performs one iteration of the training. Should store a loss value in self.loss .""" 
    #is new param vector better than last one? compare score residual, if better keep, otherwise discard.
    old_score = self.loss[-1] 
    old_theta = self.theta.copy()
    self.theta += np.random.normal(size=len(self.theta)) #replace with gradient descent.
    new_score = self.score(X,y)
    if old_score > new_score:
      self.loss.append(new_score)
    else:
      self.theta = old_theta

  def predict(self, X): 
    """Predicts outputs y from some inputs X""" 
    #pred = np.vstack(np.linspace(0, 0, num=len(X)))
    pred = np.zeros(len(X))
    for i in range(self.n_order+1):
      pred += X.ravel()**i*self.theta[i]
    return pred

  #hinge loss
  def score(self, X, y):
    ...
  
  """_summary_
  transform x into a sparse BoW array. implementation part 2
  """
  def transform(self, data):
    sparse_data = []
    for review in data:
        review_list = review.split()            #replace with tokenizer if needed
        sparse_review = np.zeros(len(review_list))
        for word in review_list:
            sparse_review[self.index_dict[word]] = 1
    sparse_data.append(sparse_review)
            
    assert len(data) == len(sparse_data)        #is the output array the same length as input?
    assert sum(sparse_data[0]) != 0             #do these sparse arrays contain anything?
    
    return np.array(sparse_data)
  
  """_summary_
  grab data from directory paths and lists
  """
  def get_data(self, dir_list, dir_path, review_polarity):
    words = set()
    reviews = []
    sentiment = []
    
    for file_name in dir_list:
        if isinstance(file_name, str):
            f = open(os.path.join(dir_path, file_name),'r')
            review = f.read()
            reviews.append(review)
            words = words|set(review.split()) #.strip("") for future comparison
            sentiment.append(review_polarity)
            f.close()
    return np.array(reviews), np.array(sentiment), words

#Part1
**Implementation task:** Implement a parser for the dataset. The output should be a list/array of strings (`X_raw`) and a list/array of labels (`y`) encoded as {-1,1}.

In [ ]:
!wget -N http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

In [2]:
def read_tar():
    # open file 
    review_file = tarfile.open('review_polarity.tar.gz') 
  
    # extracting file 
    review_file.extractall('.') 
  
    review_file.close() 

In [7]:
#shuffles both predictions and raw data with same permutation.
def shuffle(data, labels):
    p = np.random.permutation(len(data))
    return data[p], labels[p]

In [8]:
X_raw, y = shuffle(X_raw, y)

dimensions look good! now to move onto implementing BOW

#Part2

**Implementation task:** You should re-implement the feature extraction above. The list/array called `ordered_vocabulary` should contain the words for each feature dimension, and X should contain the BOW binary vectors. Remember to use the same method names as the original sklearn class.

In [9]:
index_dict = dict()
for i, word in enumerate(sorted_words):
  index_dict[word] = i

for word in ['dolphin', 'the', 'coffee']:
  if word in words:
    print("'%s' is represented as feature dimension %i" %(word, index_dict[word]))
  else:
    print("'%s' is not in the vocabulary" % word)

'dolphin' is represented as feature dimension 13868
'the' is represented as feature dimension 45372
'coffee' is represented as feature dimension 9677


In [15]:
vocabulary = set(sorted_words)      #I will take it as a given this will be useful.

In [10]:
X = self.transform(X_raw)            #output should be bag of words rep of sentence

TypeError: transform() missing 1 required positional argument: 'data'

#Part3
**Implementation task:** You should implement your versions of the following parts (you can also find this in the slides):

from sklearn.linear_model import SGDClassifier

"Set hyperparameters (these variables are only here for clarity)"
reguliser_dampening = 0.001   # lambda
learning_rate = .1            # gamma

"Create the untrained classifier"
model = SGDClassifier(loss='hinge', penalty='l2',
                      alpha=reguliser_dampening, verbose=1,
                      learning_rate='constant', eta0=learning_rate)

"Train the classifier"
model.fit(X, y)

"Get the parameter vector"
omega = np.concatenate([model.intercept_, model.coef_.ravel()])

#Part4
**Implementation task:** Implement code for printing a sorted table of your sampled hyperparameters. Note, you do not have to reimplement the grid search.